In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

In [9]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including SBERT embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize the SentenceTransformer model if it hasn't been already
    model = SentenceTransformer('all-mpnet-base-v2') # Example model, you can change this

    # Generate embeddings for the text column
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)
    # Convert embeddings to DataFrame and assign string column names
    sbert_df = pd.DataFrame(embeddings, columns=[f'sbert_{i}' for i in range(embeddings.shape[1])]) # Assign string column names

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], sbert_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)  # Convert all column names to strings

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'label' instead of 'target'
y = df['confidence'].apply(lambda x: x + 1)  # Change 'target' to the correct column name

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Number of components selected: 2
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 11}
KNN Model Metrics:
Train Accuracy: 0.9945
Train Precision: 0.9945
Train Recall: 0.9945
Train F1 Score: 0.9945
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   1 365]]

Test Accuracy: 0.6569
Test Precision: 0.5823
Test Recall: 0.6569
Test F1 Score: 0.6160
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  8 19]
 [ 1 14 82]]

Running Random Forest...
Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9357
Train Precision: 0.9393
Train Recall: 0.9357
Train F1 Score: 0.9296
Train Confusion Matrix:
[[ 14   3  15]
 [  0 130  16]
 [  0   1 365]]

Test Accuracy: 0.6423
Test Precision: 0.5343
Test Recall: 0.6423
Test F1 Score: 0.5796
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  3 24]
 [ 0 12 85]]

Running Decision Tree...
Best Params: {'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 40}
Decision Tree Model Metr

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.6746
Train Precision: 0.5890
Train Recall: 0.6746
Train F1 Score: 0.5493
Train Confusion Matrix:
[[  0   1  31]
 [  0   2 144]
 [  0   1 365]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running XGBoost...
Best Params: {'subsample': 0.8, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9890
Train Precision: 0.9891
Train Recall: 0.9890
Train F1 Score: 0.9889
Train Confusion Matrix:
[[ 31   0   1]
 [  0 141   5]
 [  0   0 366]]

Test Accuracy: 0.6058
Test Precision: 0.5323
Test Recall: 0.6058
Test F1 Score: 0.5662
Test Confusion Matrix:
[[ 0  3 10]
 [ 0  4 23]
 [ 1 17 79]]

Running AdaBoost...


...

Best Params: {'learning_rate': 0.3, 'iterations': 100, 'depth': 4}
CatBoost Model Metrics:
Train Accuracy: 0.7776
Train Precision: 0.8015
Train Recall: 0.7776
Train F1 Score: 0.7366
Train Confusion Matrix:
[[  2   5  25]
 [  0  61  85]
 [  0   6 360]]

Test Accuracy: 0.6350
Test Precision: 0.5326
Test Recall: 0.6350
Test F1 Score: 0.5757
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  3 24]
 [ 1 12 84]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6765
Train Precision: 0.7227
Train Recall: 0.6765
Train F1 Score: 0.5496
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 144]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]



In [10]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including SBERT embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize the SentenceTransformer model if it hasn't been already
    model = SentenceTransformer('all-mpnet-base-v2') # Example model, you can change this

    # Generate embeddings for the text column
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)
    # Convert embeddings to DataFrame and assign string column names
    sbert_df = pd.DataFrame(embeddings, columns=[f'sbert_{i}' for i in range(embeddings.shape[1])]) # Assign string column names

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], sbert_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)  # Convert all column names to strings

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'label' instead of 'target'
y = df['confidence'].apply(lambda x: x + 1)  # Change 'target' to the correct column name

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Number of components selected: 2
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 13}
KNN Model Metrics:
Train Accuracy: 0.9926
Train Precision: 0.9926
Train Recall: 0.9926
Train F1 Score: 0.9926
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   2 364]]

Test Accuracy: 0.6423
Test Precision: 0.6538
Test Recall: 0.6423
Test F1 Score: 0.6070
Test Confusion Matrix:
[[ 1  1 11]
 [ 0  6 21]
 [ 0 16 81]]

Running Random Forest...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 20}
Random Forest Model Metrics:
Train Accuracy: 0.9926
Train Precision: 0.9926
Train Recall: 0.9926
Train F1 Score: 0.9926
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   2 364]]

Test Accuracy: 0.5766
Test Precision: 0.5541
Test Recall: 0.5766
Test F1 Score: 0.5567
Test Confusion Matrix:
[[ 1  3  9]
 [ 0  4 23]
 [ 2 21 74]]

Running Decision Tree...
Best Params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30}
Decision Tree Model Metri

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'tanh'}
MLP Model Metrics:
Train Accuracy: 0.6746
Train Precision: 0.5885
Train Recall: 0.6746
Train F1 Score: 0.5460
Train Confusion Matrix:
[[  0   1  31]
 [  0   1 145]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running XGBoost...
Best Params: {'subsample': 0.5, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9026
Train Precision: 0.9089
Train Recall: 0.9026
Train F1 Score: 0.8942
Train Confusion Matrix:
[[ 12   2  18]
 [  0 118  28]
 [  0   5 361]]

Test Accuracy: 0.6131
Test Precision: 0.5418
Test Recall: 0.6131
Test F1 Score: 0.5750
Test Confusion Matrix:
[[ 0  3 10]
 [ 0  5 22]
 [ 0 18 79]]

Running AdaBoost...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.8805
Train Precision: 0.8888
Train Recall: 0.8805
Train F1 Score: 0.8697
Train Confusion Matrix:
[[ 11   4  17]
 [  0 107  39]
 [  0   5 361]]

Test Accuracy: 0.6350
Test Precision: 0.5521
Test Recall: 0.6350
Test F1 Score: 0.5897
Test Confusion Matrix:
[[ 0  3 10]
 [ 0  5 22]
 [ 0 15 82]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6765
Train Precision: 0.7227
Train Recall: 0.6765
Train F1 Score: 0.5496
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 144]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]



In [11]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including SBERT embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize the SentenceTransformer model if it hasn't been already
    model = SentenceTransformer('all-mpnet-base-v2') # Example model, you can change this

    # Generate embeddings for the text column
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)
    # Convert embeddings to DataFrame and assign string column names
    sbert_df = pd.DataFrame(embeddings, columns=[f'sbert_{i}' for i in range(embeddings.shape[1])]) # Assign string column names

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], sbert_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)  # Convert all column names to strings

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'


# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'label' instead of 'target'
y = df['confidence'].apply(lambda x: x + 1)  # Change 'target' to the correct column name

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Number of components selected: 2
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 13}
KNN Model Metrics:
Train Accuracy: 0.9926
Train Precision: 0.9926
Train Recall: 0.9926
Train F1 Score: 0.9926
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   2 364]]

Test Accuracy: 0.6423
Test Precision: 0.6538
Test Recall: 0.6423
Test F1 Score: 0.6070
Test Confusion Matrix:
[[ 1  1 11]
 [ 0  6 21]
 [ 0 16 81]]

Running Random Forest...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9357
Train Precision: 0.9385
Train Recall: 0.9357
Train F1 Score: 0.9304
Train Confusion Matrix:
[[ 15   3  14]
 [  0 131  15]
 [  0   3 363]]

Test Accuracy: 0.6277
Test Precision: 0.6417
Test Recall: 0.6277
Test F1 Score: 0.5942
Test Confusion Matrix:
[[ 1  2 10]
 [ 0  5 22]
 [ 0 17 80]]

Running Decision Tree...
Best Params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30}
Decision Tree Model Metri

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.6746
Train Precision: 0.6328
Train Recall: 0.6746
Train F1 Score: 0.5487
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 144]
 [  0   1 365]]

Test Accuracy: 0.6934
Test Precision: 0.4982
Test Recall: 0.6934
Test F1 Score: 0.5799
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  2 95]]

Running XGBoost...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.8805
Train Precision: 0.8888
Train Recall: 0.8805
Train F1 Score: 0.8697
Train Confusion Matrix:
[[ 11   4  17]
 [  0 107  39]
 [  0   5 361]]

Test Accuracy: 0.6350
Test Precision: 0.5521
Test Recall: 0.6350
Test F1 Score: 0.5897
Test Confusion Matrix:
[[ 0  3 10]
 [ 0  5 22]
 [ 0 15 82]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6765
Train Precision: 0.7227
Train Recall: 0.6765
Train F1 Score: 0.5496
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 144]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]



In [12]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including SBERT embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize the SentenceTransformer model if it hasn't been already
    model = SentenceTransformer('all-mpnet-base-v2') # Example model, you can change this

    # Generate embeddings for the text column
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)
    # Convert embeddings to DataFrame and assign string column names
    sbert_df = pd.DataFrame(embeddings, columns=[f'sbert_{i}' for i in range(embeddings.shape[1])]) # Assign string column names

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], sbert_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)  # Convert all column names to strings

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)

input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'


# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'label' instead of 'target'
y = df['confidence'].apply(lambda x: x + 1)  # Change 'target' to the correct column name

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Number of components selected: 2
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 11}
KNN Model Metrics:
Train Accuracy: 0.9945
Train Precision: 0.9945
Train Recall: 0.9945
Train F1 Score: 0.9945
Train Confusion Matrix:
[[ 32   0   0]
 [  0 144   2]
 [  0   1 365]]

Test Accuracy: 0.6569
Test Precision: 0.5823
Test Recall: 0.6569
Test F1 Score: 0.6160
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  8 19]
 [ 1 14 82]]

Running Random Forest...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9945
Train Precision: 0.9945
Train Recall: 0.9945
Train F1 Score: 0.9945
Train Confusion Matrix:
[[ 32   0   0]
 [  0 143   3]
 [  0   0 366]]

Test Accuracy: 0.6496
Test Precision: 0.5479
Test Recall: 0.6496
Test F1 Score: 0.5905
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  4 23]
 [ 0 12 85]]

Running Decision Tree...
Best Params: {'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': None}
Decision Tree Model M

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running XGBoost...


...

Best Params: {'learning_rate': 0.3, 'iterations': 100, 'depth': 4}
CatBoost Model Metrics:
Train Accuracy: 0.7776
Train Precision: 0.8015
Train Recall: 0.7776
Train F1 Score: 0.7366
Train Confusion Matrix:
[[  2   5  25]
 [  0  61  85]
 [  0   6 360]]

Test Accuracy: 0.6350
Test Precision: 0.5326
Test Recall: 0.6350
Test F1 Score: 0.5757
Test Confusion Matrix:
[[ 0  1 12]
 [ 0  3 24]
 [ 1 12 84]]

Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6728
Train Precision: 0.4527
Train Recall: 0.6728
Train F1 Score: 0.5412
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 146]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.6765
Train Precision: 0.7227
Train Recall: 0.6765
Train F1 Score: 0.5496
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 144]
 [  0   0 366]]

Test Accuracy: 0.7080
Test Precision: 0.5013
Test Recall: 0.7080
Test F1 Score: 0.5870
Test Confusion Matrix:
[[ 0  0 13]
 [ 0  0 27]
 [ 0  0 97]]

